# ch07-종합을 위한 그룹화, 필터링, 그리고 변환

- 데이터 분석 중 가장 기본적인 과제는 각 그룹에 대한 계산을 수행하기 전에 데이터를 독립적인 그룹으로 분할하는 것이다.
- 분할-적용-병합 split-apply-combine 으로 불린다.
- groupby() 메서드의 결과는 groupby 객체다.
- groupby 객체가 생성될 때 pandas가 하는 일을 별로 없고 단지 주어진 조건의 그룹화가 가능한지 확인만 한다.
- 제대로 사용하려면 groupby 객체에 메서드를 체인시켜야 한다.

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## 1.종합 aggregation에 대한 정의

- groupby() 메서드의 가장 보편적인 사용법은 종합 aggregation이다.
- 데이터 분석에서 aggregation 이란 여러 입력이 합산되거나 병합되어 단일 출력값으로 만들어지는 것을 의미한다.
- col의 모든 값을 더하거나 최대값을 찾는 것이 연속된 단일 데이터에 aggregation 을 적용한 예다.
- 주요한 aggregation 함수로는 sum, min, max, mean, count, variance, std 등이 있다.

- 비행 데이터셋을 조사하고 가장 간단한 형태일 단일 그룹화 col, 단일 col aggregation, 단일 aggregation 함수만 사용하는 aggregation을 수행해본다.
- 각 항공사의 평균 연착율을 알아보자.

- 항공 데이터셋을 읽어들인다.

In [2]:
flights = pd.read_csv('../data/flights.csv')

In [3]:
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


- groupby() 메서드에 그룹화할 col을 전달하고 agg 메서드에 aggregation 할 col과 aggregation 함수를 딕셔너리 형태로 전달하여 호출한다.

In [4]:
flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'}).head()

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


- 또 다른 방법으로는 종합할 col을 index operator에 놓고 aggregation 함수를 agg의 문자열로 전달할 수도 있다.

In [5]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

- 함수 이름을 문자열로 사용할 수 있는 것은 특정 aggregation 함수를 문자열로 참고할 수 있도록 pandas에 구현되어 있는 기능이다.
- numpy의 mean 함수를 사용한 방법처럼 어떤 aggregation 함수라도 직접 agg 메서드에 전달할 수 있다.

In [6]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.mean).head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

- 이 예제의 경우 agg() 메서드를 쓰지 않고 바로 mean() 메서드를 사용할 수도 있다.

In [7]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

- groupby() 메서드의 결과에 메서드 체인을 하는 대신 그 결과 자체를 변수에 저장해보자.

In [8]:
grouped = flights.groupby('AIRLINE')

In [9]:
type(grouped)

pandas.core.groupby.DataFrameGroupBy

- 이 단계에서는 어떤한 계산도 일어나지 않는다.
- pandas는 단지 그룹화된 col을 검증하지만 한다.
- groupby 객체는 aggregation 연산을 수행하기 위한 agg() 메서드를 갖고 있다.
- 이 메서드를 사용하는 방법 중 하나는 앞에서 본 것처럼 이 객체에 aggregation col을 aggregation 함수로 매핑하는 딕셔너리를 전달하는 것이다.

- 몇 가지 다른 문법을 사용할 수 있다.
- 딕셔너리 대신 DataFrame에서 col을 선택하는 것처럼 index operator를 사용할 수도 있다.
- 그 경우 함수의 문자열 이름이 agg() 메서드에 스칼라 값으로 잔달된다.

- 단일 aggregation 함수만 적용하면 agg() 메서드 없이 groupby 객체 자체의 메서드를 직접 호출할 수 있다.
- 다음에 열거된 aggregation 함수들은 agg() 메서드에 문자열로 전달하거나 groupby 객체에 메서드로 직접 사용할 수 있다.
    - min
    - max
    - mean
    - median
    - sum
    - count
    - std
    - var
    - size
    - describe
    - nunique
    - idxmin
    - idxmax

- agg() 메서드에 aggregation 함수가 아닌 것을 사용하면 ValueError가 발생한다.

## 2.복수 col과 함수를 사용한 그룹화와 aggregation

- 복수 개의 col을 사용해도 그룹화와 aggregation을 할 수 있다.
- 단지 단일 col을 사용할 때와 문법만 조금 다르다.
- 그룹화 연산과 마찬가지로 그룹화 col, aggregation col, aggregation 함수의 세 가지 구성 요소를 따로 구분해두는 것이 좋다.

- 다음 항목을 구해보자.
    - 각 항공사별 주 중 운항 취소 건수
    - 각 항공사별로 주 중 취소 건수, 우회 운항 건수와 비율
    - 각 출발지와 목적지별 전체 운향 편수, 운향 취소된 건수와 비율, 비행 시간의 평균과 분산

- 각 항공사별 주 중 운항 취소 건수
- 항공 데이터셋을 읽어 들인 후, AIRLINE, WEEKDAY를 그룹화하고
- CANCELLED col을 aggregation 한 후 sum 한다.

In [10]:
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].agg(np.sum).head(7)

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
         6          21
         7          29
Name: CANCELLED, dtype: int64

- 각 항공사별로 주 중 취소 건수, 우회 운항 건수와 비율

In [11]:
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED', 'DIVERTED'].agg([np.sum, np.mean]).head(7)

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
        6              21  0.018667        9  0.008000
        7              29  0.021837        1  0.000753

- 각 출발지와 목적지별 전체 운향 편수, 운향 취소된 건수와 비율, 비행 시간의 평균과 분산

In [12]:
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [13]:
group_cols = ['ORG_AIR', 'DEST_AIR']

In [14]:
agg_dict = {'CANCELLED': ['sum', 'mean', 'size'],
            'AIR_TIME': ['mean', 'var']}

In [15]:
flights.groupby(group_cols).agg(agg_dict).head()

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

- 복수 개의 col을 그룹화하려면 groupby() 메서드에 문자열 이름을 리스트로 전달한다.
- AIRLINE, WEEKDAY의 각각의 고유한 조합이 서로 독립된 그룹을 형성한다.
- size aggregation 함수는 그룹별로 전체 row 개수를 반환한다.
- 그룹별로 누락되지 않은 개수를 반환하는 count와는 다른 작동 방식이다.

- 다음 4개의 유사 코드 블록이 groupby() 메서드를 사용해 aggregation을 사용하는 주요 방법이다.

- 1.agg를 딕셔너리와 사용하면 가장 유연하며 각 열별로 종합 함수를 설정할 수 있게 해준다.
- ```df.groupby(['grouping', 'columns'])\
     .agg({'agg_col1':['list', 'of', 'functions'],
     'agg_col2':['other', 'functions']})```

- 2.agg를 aggregation 함수의 리스트와 사용하면 각 함수를 각각의 aggregation 열에 적용시킨다.
- `df.groupby(['grouping', 'columns'])['aggregating', 'columns']).agg([aggregating, function])`

- 3.agg를 사용하는 대신 aggregation col다음에 직접적으로 메서드를 사용하는 방식은 각각의 aggregation col에 대해서만 사용할 수 있다.
- 이 방식은 복수 개의 aggregation 함수를 사용할 수 없다.
- `df.groupby(['grouping', 'columns'])['aggregating', 'columns']).aggregating_method()`

- 4.aggregation col을 따로 설정하지 않으면 aggregation 메서드는 그룹화되지 않은 모든 열에 대해 적용된다.
- `df.groupby(['grouping', 'columns']).aggregatiing_method()`

## 3.그룹화 후 MultiIndex 제거

- groupby를 사용하다 보면 col이나 row 혹은 둘 다에 대해 MultiIndex가 생성되기 마련이다.
- 복수 인덱스를 가진 DataFrame의 경우, 검색이 훨씬 까다롭고 혼란스럽다.
- 적절한 조작을 통해 인덱스를 단일 레벨로 조정하고 col 이름을 알아보기 쉽도록 처리하자.

- 항공 데이터셋을 읽어 들인 후, 운항 전체 마일 수와 평균을 찾는다.
- 그 다음 각 항공사별로 각 주 중의 최대와 최소 연착을 찾아낸다.

In [16]:
flights = pd.read_csv('../data/flights.csv')

In [17]:
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [18]:
airline_info = flights.groupby(['AIRLINE', 'WEEKDAY'])\
                      .agg({'DIST': ['sum', 'mean'],
                            'ARR_DELAY': ['min', 'max']})\
                      .astype(int)

In [19]:
airline_info.head(7)

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
        6        1265340  1124       -50  858
        7        1461906  1100       -49  626

- col, row 모두 2단계의 MultiIndex로 레이블된다.
- 이것을 한 단계로 압축해보자.
- col을 압축하려면 get_level_values라는 MultiIndex 메서드를 사용해야 한다.

In [20]:
level0 = airline_info.columns.get_level_values(0)

In [21]:
level0

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [22]:
level1 = airline_info.columns.get_level_values(1)

In [23]:
level1

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [24]:
airline_info.columns = level0 + '_' + level1

In [25]:
airline_info.head(7)

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139            -60            551
        2         1358256       1107            -52            725
        3         1496665       1117            -45            473
        4         1452394       1089            -46            349
        5         1427749       1122            -41            732
        6         1265340       1124            -50            858
        7         1461906       1100            -49            626

- reset_index() 메서드를 사용해 row 레이블을 단일 레이블을 변경한다.

In [26]:
airline_info.reset_index().head(7)

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
5,AA,6,1265340,1124,-50,858
6,AA,7,1461906,1100,-49,626


- pandas는 디폴트로 groupby 연산의 마지막에 모든 그룹화 열을 인덱스로 만든다.
- groupbu 메서드의 as_index 매개변수를 False로 설정하면 이 작동 방법을 바꿀 수 있다.

- 각 항공사별로 운항 거리의 평균을 찾아보자.

In [27]:
flights.groupby(['AIRLINE'], as_index=False)['DIST'].agg('mean').round(0)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
5,F9,970.0
6,HA,2615.0
7,MQ,404.0
8,NK,1047.0
9,OO,511.0


- pandas는 디폴트로 그룹화된 col을 정렬한다.
- groupby() 메서드 내 sort 매개변수 값의 디폴트 값은 True다.
- 정렬을 건너뛴다면 약간의 성능 향상을 기대할 수 있다.

## 4.종합 함수 커스터마이즈

- 대락 데이터셋을 사용하여 각 state 별로 학부생 인원의 평균과 표준편차를 확인해보자.

In [28]:
college = pd.read_csv('../data/college.csv')

In [29]:
college.groupby('STABBR')['UGDS'].agg(['mean', 'std']).round(0).head()

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0


- 평균으로부터 가장 멀리 떨어져 있는 표준편차를 구하려면?
- 이 값을 구하려면 전체 학부생 인원의 평균을 각 학교의 학부생 인원에서 차감한 후 표준편차로 나눠야 한다.
- 이를 통해 각 그룹의 학부생 인원이 표준화된다.
- 이 값의 절대값이 가장 큰 것을 찾으면 그 값이 평균으로부터 가장 멀리 떨어진 값이 된다.

In [30]:
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

- 정의한 함수를 agg() 메서드에 전달한다.

In [31]:
college.groupby('STABBR')['UGDS'].agg(max_deviation).round(1).head()

STABBR
AK    2.6
AL    5.8
AR    6.3
AS    NaN
AZ    9.9
Name: UGDS, dtype: float64

- max_deviation 함수는 단일 매개변수 s를 입력으로 받는다.
- pandas에서는 묵시적으로 UGDS col을 max_deviation에 Series로 전달한다.

- 직접 만든 함수를 복수의 aggregation col에 적용할 수도 있다.

In [32]:
college.groupby('STABBR')['UGDS', 'SATVRMID', 'SATMTMID'].agg(max_deviation).round(1).head()

,UGDS,SATVRMID,SATMTMID
STABBR,,,
AK,2.6,NaN,NaN
AL,5.8,1.6,1.8
AR,6.3,2.2,2.3
AS,NaN,NaN,NaN
AZ,9.9,1.9,1.4


- 직접만든 aggregation 함수를 기존 함수와 함께 사용할 수도 있다.

In [33]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID']\
       .agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                      SATVRMID               \
                max_deviation    mean     std max_deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                  2.1  3508.9  4539.5           NaN    NaN   NaN   
       1                  1.1   123.3   132.9           NaN  555.0   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  514.9  56.5   
       1                  2.4   979.7   870.8           1.5  498.0  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  481.1  37.9   

                     SATMTMID               
                max_deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0

- pandas는 반환된 col 이름에 함수 이름을 붙여 사용한다.
- col 이름은 rename 메서드를 사용해 직접적으로 바꾸거나 `__name__`을 사용해 변경할 수 있다.

In [34]:
max_deviation.__name__

'max_deviation'

In [35]:
max_deviation.__name__ = 'Max Deviation'

In [36]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID']\
       .agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                      SATVRMID               \
                Max Deviation    mean     std Max Deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                  2.1  3508.9  4539.5           NaN    NaN   NaN   
       1                  1.1   123.3   132.9           NaN  555.0   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  514.9  56.5   
       1                  2.4   979.7   870.8           1.5  498.0  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  481.1  37.9   

                     SATMTMID               
                Max Deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0

## 5.*arg, **kwargs 함수를 사용한 종합 커스터마이즈

- 사용자 정의 aggregation 함수를 작성할 경우 pandas는 묵시적으로 각 aggregation col을 한 번에 하나씩 Series 형식으로 전달한다.
- 함수에 Series 자체보다 더 많은 인수를 전달해야 할 경우가 있다.
- inspect 모듈을 사용해 groupby 객체의 agg 메서드에 대한 signature를 확인해보자.

In [37]:
college = pd.read_csv('../data/college.csv')

In [38]:
grouped_ = college.groupby(['STABBR', 'RELAFFIL'])

In [39]:
import inspect
inspect.signature(grouped.agg)

<Signature (arg, *args, **kwargs)>

- 대학 데이터셋에서 주와 종교에 대해 지정한 두 값 사이의 학부생이 존재하는 학교의 비율을 찾아본다.

- 학부생 인원이 1000명에서 3000명 사이에 있는 학교의 비율을 반환하는 함수를 정의한다.

In [40]:
def pct_between_1_3k(s):
    return s.between(1000, 3000).mean()

In [41]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS']\
       .agg(pct_between_1_3k).head(9)

STABBR  RELAFFIL
AK      0           0.142857
        1           0.000000
AL      0           0.236111
        1           0.333333
AR      0           0.279412
        1           0.111111
AS      0           1.000000
AZ      0           0.096774
        1           0.000000
Name: UGDS, dtype: float64

- 상한과 하한값을 조정할 수 있도록 개선해보자.

In [42]:
def pct_between(s, low, high):
    return s.between(low, high).mean()

In [43]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS']\
       .agg(pct_between, 1000, 10000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

- 다음과 같이 매개변수 이름을 명시적으로 사용할 수도 있다.

In [44]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS']\
       .agg(pct_between, high=10000, low=1000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

- pandas는 복수 개의 aggregation 함수를 동시에 사용할 경우 추가 인수를 전달할 수 있는 직접적인 방법이 제공되지 않는다.
- 커스터마이즈된 함수를 다른 내장 함수 등과 같이 사용하려면 closure를 사용해야 한다.|

In [45]:
def make_agg_func(func, name, *args, **kwargs):
    def wrapper(x):
        return func(x, *args, **kwargs)
    wrapper.__name__ = name
    return wrapper

In [46]:
my_agg1 = make_agg_func(pct_between, 'pct_1_3k', low=1000, high=3000)

In [47]:
my_agg2 = make_agg_func(pct_between, 'pct_10_30k', 10000, 30000)

In [48]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS']\
       .agg(['mean', my_agg1, my_agg2]).head()

mean  pct_1_3k  pct_10_30k
STABBR RELAFFIL                                   
AK     0         3508.857143  0.142857    0.142857
       1          123.333333  0.000000    0.000000
AL     0         3248.774648  0.236111    0.083333
       1          979.722222  0.333333    0.000000
AR     0         1793.691176  0.279412    0.014706

## 6. groupby 객체 조사

- groupby() 메서드를 DataFrame에 적용하면 그 결과는 groupby 객체다.
- aggregation이나 변환을 위해 이 객체를 별도로 변수에 저장하지 않고 계속 연산을 사용한다.
- groupby() 객체를 조사하는 가장 큰 목적 중 하나는 개별 그룹을 살펴보기 위함이다.

- 대학 데이터셋을 읽어 주와 종교 col을 그룹화한다.

In [49]:
college = pd.read_csv('../data/college.csv')

In [50]:
grouped = college.groupby(['STABBR', 'RELAFFIL'])

In [51]:
type(grouped)

pandas.core.groupby.DataFrameGroupBy

- dir 함수를 사용해 함수를 확인해보자.

In [52]:
print([attr for attr in dir(grouped) if not attr.startswith('_')])

['CITY', 'CURROPER', 'DISTANCEONLY', 'GRAD_DEBT_MDN_SUPP', 'HBCU', 'INSTNM', 'MD_EARN_WNE_P10', 'MENONLY', 'PCTFLOAN', 'PCTPELL', 'PPTUG_EF', 'RELAFFIL', 'SATMTMID', 'SATVRMID', 'STABBR', 'UG25ABV', 'UGDS', 'UGDS_2MOR', 'UGDS_AIAN', 'UGDS_ASIAN', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_NHPI', 'UGDS_NRA', 'UGDS_UNKN', 'UGDS_WHITE', 'WOMENONLY', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


- ngroups attr 를 사용하여 그룹 개수를 알아보자.

In [53]:
grouped.ngroups

112

- 각 그룹을 고유하게 구분하는 레이블을 찾기 위해 각 고유한 그룹에 해당하는 인덱스 레이블을 매핑한 딕셔너리 값을 갖고 있는 groups attr를 확인해보자.

In [54]:
groups = list(grouped.groups.keys())

In [55]:
groups[:6]

[('AK', 0), ('AK', 1), ('AL', 0), ('AL', 1), ('AR', 0), ('AR', 1)]

- get_group() 메서드에 정확한 그룹 레이블을 튜플로 전달해 단일 그룹을 추출할 수 있다.
- 플로리다 주에서 종교와 연계된 모든 학교를 찾아보자.

In [56]:
grouped.get_group(('FL', 1)).head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
712,The Baptist College of Florida,Graceville,FL,0.0,0.0,0.0,1,545.0,465.0,0.0,...,0.0308,0.0000,0.0507,0.2291,1,0.5878,0.5602,0.3531,30800,20052
713,Barry University,Miami,FL,0.0,0.0,0.0,1,470.0,462.0,0.0,...,0.0164,0.0741,0.0841,0.1518,1,0.5045,0.6733,0.4361,44100,28250
714,Gooding Institute of Nurse Anesthesia,Panama City,FL,0.0,0.0,0.0,1,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,PrivacySuppressed
715,Bethune-Cookman University,Daytona Beach,FL,1.0,0.0,0.0,1,405.0,395.0,0.0,...,0.0198,0.0205,0.0190,0.0523,1,0.7758,0.8867,0.0647,29400,36250
724,Johnson University Florida,Kissimmee,FL,0.0,0.0,0.0,1,480.0,470.0,0.0,...,0.0045,0.0045,0.0136,0.1636,1,0.6689,0.7384,0.2185,26300,20199


- 각 개별 그룹을 살펴보자.
- groupby 객체가 iterable 하기 때문에 가능하다.

In [57]:
from IPython.display import display
for name, group in grouped:
    print(name)
    display(group.head(3))

('AK', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0980,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
62,University of Alaska Fairbanks,Fairbanks,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0401,0.0110,0.3060,0.3887,1,0.2263,0.2550,0.4519,36200,19355
63,University of Alaska Southeast,Juneau,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0686,0.0049,0.2241,0.5112,1,0.1769,0.1996,0.5550,37400,16875


('AK', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
61,Alaska Bible College,Palmer,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0370,0.0,0.0000,0.1481,1,0.3571,0.2857,0.4286,NaN,PrivacySuppressed
64,Alaska Pacific University,Anchorage,AK,0.0,0.0,0.0,1,555.0,503.0,0.0,...,0.0945,0.0,0.0873,0.3745,1,0.3152,0.5297,0.4910,47000,23250
5417,Alaska Christian College,Soldotna,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0147,0.0,0.1324,0.0735,1,0.8868,0.6792,0.2264,NaN,PrivacySuppressed


('AL', 0)


...

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
2842,Scott College of Cosmetology,Wheeling,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0519,0.0,0.0,0.0000,1,0.5276,0.4961,0.1111,14800,9250
4019,B M Spurr School of Practical Nursing,Glen Dale,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0000,0.0,0.0,0.0000,1,0.4722,0.0000,0.4444,PrivacySuppressed,PrivacySuppressed
4020,Ben Franklin Career Center,Dunbar,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0023,0.0,0.0,0.7885,1,0.1761,0.0000,0.7568,20800,PrivacySuppressed


('WV', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
4016,Alderson Broaddus University,Philippi,WV,0.0,0.0,0.0,1,475.0,485.0,0.0,...,0.0098,0.0342,0.0020,0.0342,1,0.4800,0.8945,0.0722,46000,27000
4018,Appalachian Bible College,Mount Hope,WV,0.0,0.0,0.0,1,514.0,487.0,0.0,...,0.0000,0.0051,0.0000,0.0612,1,0.4388,0.2770,0.0899,28700,9300
4027,Davis & Elkins College,Elkins,WV,0.0,0.0,0.0,1,485.0,490.0,0.0,...,0.0000,0.1466,0.1657,0.0167,1,0.4188,0.6579,0.1133,35000,23840.5


('WY', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
4128,Casper College,Casper,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0163,0.0078,0.0269,0.3250,1,0.2188,0.1455,0.3447,34800,10764
4129,Central Wyoming College,Riverton,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0300,0.0073,0.0173,0.4377,1,0.1700,0.0967,0.3992,25200,8757
4130,Eastern Wyoming College,Torrington,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0088,0.0015,0.0000,0.3275,1,0.1729,0.1166,0.2371,25900,10000


- head() 메서드를 groupby 객체에 사용하면 각 그룹의 처음 행을 한꺼번에 단일 DataFrame으로 얻을 수 있다.

In [58]:
grouped.head(2).head(6)

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
10,Birmingham Southern College,Birmingham,AL,0.0,0.0,0.0,1,560.0,560.0,0.0,...,0.0051,0.0000,0.0051,0.0017,1,0.1920,0.4809,0.0152,44200,27000
43,Prince Institute-Southeast,Elmhurst,IL,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0000,0.0000,0.0000,0.0000,1,0.7857,0.9375,0.6569,PrivacySuppressed,20992
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0980,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5


- 모든 그룹을 반복하는 작업은 드물고, 가능하다면 피해야한다.
- 작업이 상당히 느릴 수 있다.
- groupby 객체를 반복하면 그룹 이름과 그룹화된 열이 포함되지 않은 DataFrame으로 구성된 튜플이 주어진다.
- 이 튜플은 for 루프에서 name, group으로 분해할 수 있다.

- 그룹을 반복하면서 각 그룹의 행 몇 개를 직접 출력할 수 있다.
- 이를 위해 print 함수를 사용하거나 IPython.display 모듈의 display 함수를 사용할 수 있다.
- print 함수를 사용하면 순수 텍스트 DataFrmae 결과를 출력한다.
- display를 사용하면 보통 notebook에서 출력되는 읽기 편한 형식 출력된다.

- nth() 메서드는 정수 리스트가 주어지면 각 그룹에서 특정 위치에 해당하는 행을 선택한다.
- 다음의 결과는 처음과 마지막 행을 각 그룹에서 선택한다.

In [59]:
grouped.nth([1, -1]).head(8)

CITY  CURROPER  DISTANCEONLY GRAD_DEBT_MDN_SUPP  HBCU  \
STABBR RELAFFIL                                                                
AK     0          Fairbanks         1           0.0              19355   0.0   
       0             Barrow         1           0.0  PrivacySuppressed   0.0   
       1          Anchorage         1           0.0              23250   0.0   
       1           Soldotna         1           0.0  PrivacySuppressed   0.0   
AL     0         Birmingham         1           0.0            21941.5   0.0   
       0             Dothan         1           0.0  PrivacySuppressed   0.0   
       1         Birmingham         1           0.0              27000   0.0   
       1         Huntsville         1           NaN            36173.5   NaN   

                                                  INSTNM MD_EARN_WNE_P10  \
STABBR RELAFFIL                                                            
AK     0                  University of Alaska Fairbanks           36200   
       0                               Ilisagvik College           24900   
       1                       Alaska Pacific University           47000   
       1                        Alaska Christian College             NaN   
AL     0             University of Alabama at Birmingham           39700   
       0         Alabama College of Osteopathic Medicine             NaN   
       1                     Birmingham Southern College           44200   
       1            Strayer University-Huntsville Campus           49200   

                 MENONLY  PCTFLOAN  PCTPELL    ...      UGDS_2MOR  UGDS_AIAN  \
STABBR RELAFFIL                                ...                             
AK     0             0.0    0.2550   0.2263    ...         0.0401     0.1284   
       0             0.0    0.0000   0.1323    ...         0.0000     0.6881   
       1             0.0    0.5297   0.3152    ...         0.0945     0.1855   
       1             0.0    0.6792   0.8868    ...         0.0147     0.7794   
AL     0             0.0    0.5214   0.3460    ...         0.0368     0.0022   
       0             0.0       NaN      NaN    ...            NaN        NaN   
       1             0.0    0.4809   0.1920    ...         0.0051     0.0102   
       1             NaN       NaN      NaN    ...            NaN        NaN   

                 UGDS_ASIAN  UGDS_BLACK  UGDS_HISP  UGDS_NHPI  UGDS_NRA  \
STABBR RELAFFIL                                                           
AK     0             0.0126      0.0210     0.0522     0.0027    0.0110   
       0             0.0826      0.0183     0.0092     0.0459    0.0183   
       1             0.0255      0.0291     0.0364     0.0109    0.0000   
       1             0.0000      0.0000     0.0147     0.0000    0.0000   
AL     0             0.0518      0.2600     0.0283     0.0007    0.0179   
       0                NaN         NaN        NaN        NaN       NaN   
       1             0.0517      0.1102     0.0195     0.0000    0.0000   
       1                NaN         NaN        NaN        NaN       NaN   

                 UGDS_UNKN  UGDS_WHITE  WOMENONLY  
STABBR RELAFFIL                                    
AK     0            0.3060      0.4259        0.0  
       0            0.0000      0.1376        0.0  
       1            0.0873      0.5309        0.0  
       1            0.1324      0.0588        0.0  
AL     0            0.0100      0.5922        0.0  
       0               NaN         NaN        0.0  
       1            0.0051      0.7983        0.0  
       1               NaN         NaN        NaN  

[8 rows x 25 columns]

## 7.소수 인종이 다수인 주 찾기

- 데이터의 전체 그룹을 True 나 False로 설정해 False 그룹만 제거할 수 있다.
- 이렇게 하려면 그룹을 groupby 메서드로 형성한 후, filter 메서드를 적용하면 된다.
- filter 메서드는 각 그룹을 유지할 것인지, 아닌지를 결정해 True나 False 값을 반환하는 함수를 인수로 받는다.

- 대학 데이터셋을 사용해 백인보다 유색 인종이 더 많은 모든 주를 찾는다.

- 대학 데이터셋을 읽은 후 주에 대해 그룹화한 후 전체 그룹 개수를 출력한다.
- 이 수는 nunique Series 메서드로 추출한 고유한 주 개수와 동일해야 한다.

In [60]:
college = pd.read_csv('../data/college.csv', index_col='INSTNM')

In [61]:
grouped = college.groupby('STABBR')

In [62]:
grouped.ngroups

59

In [63]:
college['STABBR'].nunique()

59

- grouped 변수에는 filter() 메서드가 있어서 그룹을 유지할 것인지 삭제할 것인지 구분하는 커스터마이즈된 함수를 받아들인다.
- 이 사용자 정의 함수는 묵시적으로 현재 그룹에 대한 DataFrame이 전달되고 불리언을 반환한다.
- 소수 인종 학생의 비율ㅇ을 계산하고 이 비율이 사용자가 지정한 임계값보다 클 경우 True를 반환하는 함수를 정의해보자.

In [65]:
def check_minority(df, threshold):
    minority_pct = 1 - df['UGDS_WHITE']
    total_minority = (df['UGDS'] * minority_pct).sum()
    total_ugds = df['UGDS'].sum()
    total_minority_pct = total_minority / total_ugds
    return total_minority_pct > threshold

- filter() 메서드에 check_minority 함수와 임계값을 전달한다.

In [66]:
college_filtered = grouped.filter(check_minority, threshold=.5)

In [67]:
college_filtered.head()

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,4102.0,...,0.0373,0.0,0.1026,0.4749,0,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,83.0,...,0.0241,0.0,0.3855,0.3373,0,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,25.0,...,0.0400,0.0,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,126.0,...,0.0000,0.0,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,1.0,...,0.0000,0.0,0.0000,1.0000,0,0.0000,0.0000,0.0000,118900,PrivacySuppressed


- 원본과 필터링된 항목을 비교해보자.

In [68]:
college.shape

(7535, 26)

In [69]:
college_filtered.shape

(3028, 26)

In [71]:
college['STABBR'].nunique()

59

In [72]:
college_filtered['STABBR'].nunique()

20

- groupby의 filter() 메서드는 그룹 내의 모든 행을 유지하거나 제거한다.
- 이 연산은 col 개수를 바꾸지는 않는다.
- filter() 메서드는 이러한 편집 작업을 사용자 정의 함수를 사용해 수행한다.
- 각 그룹의 전체 DataFrame을 사용자 정의 함수에 전달하여 그 결과로 각 그룹에 대한 단일 불리언 값을 반환한다.

## 8.체중 감량 내기를 통한 변환

- 두 명의 체중 감량을 4개월간 추적해 승자를 결정한다.
- 각 달의 마지막에 그 달에 가장 체중 강량 비율이 큰 사람이 승자가 된다.
- 체중 감량을 추적하기 위해 데이터를 월별, 사람별로 그룹화하고
- transform() 메서드를 사용하 시작주 부터 각 주마다의 체중 감량 비율을 찾아본다.

- weight_loss 데이터셋을 읽는다.

In [73]:
weight_loss = pd.read_csv('../data/weight_loss.csv')

In [74]:
weight_loss.head()

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283


In [76]:
weight_loss.shape

(32, 4)

In [77]:
weight_loss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
Name      32 non-null object
Month     32 non-null object
Week      32 non-null object
Weight    32 non-null int64
dtypes: int64(1), object(3)
memory usage: 680.0+ bytes


In [78]:
weight_loss.query('Month == "Jan"')

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190


- 각 달의 승자를 결정하기 위해서는 각 달의 첫 번째 주와 마지막 주의 체중 변화만 계산하면 된다.
- 주별로 정보를 갱신하고자 한다면 매주마다 그 달의 첫 번째 주부터 체중 감량을 계산해볼 수 있다.

In [79]:
def find_perc_loss(s):
    return (s - s.iloc[0]) / s.iloc[0]

In [80]:
bob_jan = weight_loss.query('Name == "Bob" and Month == "Jan"')

In [81]:
bob_jan

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
2,Bob,Jan,Week 2,288
4,Bob,Jan,Week 3,283
6,Bob,Jan,Week 4,283


In [82]:
find_perc_loss(bob_jan['Weight'])

0    0.000000
2   -0.010309
4   -0.027491
6   -0.027491
Name: Weight, dtype: float64

- 이 함수를 사람과 주에 대한 모든 단일 조합에 대해 적용하면 각 달의 첫 주와 대비한 주별 체중 감량값을 얻을 수 있다.
- 데이터를 Name과 Month에 따라 그룹화하고 transform() 메서드를 사용해 이 커스컴 함수를 적용한다.

In [85]:
pcnt_loss = weight_loss.groupby(['Name', 'Month'])['Weight']\
                       .transform(find_perc_loss)

In [100]:
pcnt_loss.shape

(32,)

In [86]:
pcnt_loss.head(8)

0    0.000000
1    0.000000
2   -0.010309
3   -0.040609
4   -0.027491
5   -0.040609
6   -0.027491
7   -0.035533
Name: Weight, dtype: float64

- transform() 메서드는 호출하는 DataFrame과 동일한 행 개수를 갖는 객체를 반환해야만 한다.
- 이 결과를 최초 DataFrame에 새로운 col로 추가해보자.
- 출력을 단순화하기 위해 bob의 처음 두 달 데이터만 선택한다.

In [88]:
weight_loss['Perc Weight Loss'] = pcnt_loss.round(3)

In [89]:
weight_loss.query('Name=="Bob" and Month in ["Jan", "Feb"]')

,Name,Month,Week,Weight,Perc Weight Loss
0,Bob,Jan,Week 1,291,0.000
2,Bob,Jan,Week 2,288,-0.010
4,Bob,Jan,Week 3,283,-0.027
6,Bob,Jan,Week 4,283,-0.027
8,Bob,Feb,Week 1,283,0.000
10,Bob,Feb,Week 2,275,-0.028
12,Bob,Feb,Week 3,268,-0.053
14,Bob,Feb,Week 4,268,-0.053


- 매달 체중 감량 비율은 리셋되야 한다.
- 중요한 것은 오직 마지막 주의 결과이므로 4주째를 선택한다.

In [90]:
week4 = weight_loss.query('Week == "Week 4"')

In [91]:
week4

,Name,Month,Week,Weight,Perc Weight Loss
6,Bob,Jan,Week 4,283,-0.027
7,Amy,Jan,Week 4,190,-0.036
14,Bob,Feb,Week 4,268,-0.053
15,Amy,Feb,Week 4,173,-0.089
22,Bob,Mar,Week 4,261,-0.026
23,Amy,Mar,Week 4,170,-0.017
30,Bob,Apr,Week 4,250,-0.042
31,Amy,Apr,Week 4,161,-0.053


- 4주째만 선택했지만 각 월의 승자를 자동으로 찾아내지는 못한다.
- pivot 메서드로 데이터를 재구성해 체중 감량 비율이 각 월별로 인접해 나타나도록 수정하자.

In [95]:
winner = week4.pivot(index='Month', columns='Name',
                     values='Perc Weight Loss')

In [96]:
winner

Name,Amy,Bob
Month,,
Apr,-0.053,-0.042
Feb,-0.089,-0.053
Jan,-0.036,-0.027
Mar,-0.017,-0.026


- 이 출력은 각 월별로 누가 승자인지 확연히 보여주지만, 자동화를 위해서는 여전히 몇 단계 작업을 더 거쳐야 한다.
- Numpy에는 where라는 이름의 벡터화된 if-then-else 함수가 있어서 Series나 불리언 배열을 다른 값으로 매핑할 수 있다.
- 승자 이름을 담을 새로운 col을 생성하고 그 달의 승자의 감량 비율을 부각시켜 보자.

In [97]:
winner['Winner'] = np.where(winner['Amy'] < winner['Bob'], 'Amy', 'Bob')

In [98]:
winner.style.highlight_min(axis=1)

Name,Amy,Bob,Winner
Month,,,
Apr,-0.053,-0.042,Amy
Feb,-0.089,-0.053,Amy
Jan,-0.036,-0.027,Amy
Mar,-0.017,-0.026,Bob


- 마지막으로 value_counts() 메서드로 승리한 월 개수를 세어 최종 점수를 반환한다.

In [99]:
winner.Winner.value_counts()

Amy    3
Bob    1
Name: Winner, dtype: int64

- 결과를 도출하는 동안 불리언 인덱싱 대신 query() 메서드를 사용해 데이터를 필터링 했다.
- query() 메서드를 사용하면 불리언 인덱싱보다 가독성이 좋아진다.

- 각 사람의 월별 체중 감량 비율을 찾기 위해서는 각 주의 체중 감소를 월 초와 비교해야 한다.
- 이 상황에서는 groupby의 transform() 메서드가 가장 적합하다.
- transform() 메서드는 함수를 매개변수 중 하나로 받아들인다.
- 이 함수는 묵시적으로 그룹화되지 않은 각 col에 전달된다.
- 혹은 이번 예제처럼 Weight col에서 한 것처럼 인덱스 연산자에서 지정할 특정 col에 전달된다.
- 이 메서드는 전달된 그룹과 동일한 길이의 값을 반환하고, 그렇지 않으면 오류가 발생한다.
- 원시 DataFrame 으로부터의 모든 값이 변환된다.
- 어떠한 aggregation이나 filtering도 일어나지 않는다.

- pivot 함수는 데이터셋을 한 col의 고유한 값을 중심축으로 새로운 col 이름으로 만들어준다.
- index 매개변수를 사용하면 pivot으로 하기를 원하지 않는 열을 지정할 수 있다.
- values 매개변수에 전달된 col들은 그 index와 columns 매개변수의 각 고유한 조합에 대해 연속적으로 쌓인다.

- pivot() 메서드는 index와 columns 매개변수의 고유한 조합이 단 한 번만 나타날 때만 작동한다.
- 고유한 조합이 둘 이상 존재하면 오류가 발생한다.
- 그런 경우에는 pivot_table을 사용해 복수값을 한꺼번에 종합할 수 있다.

- Numpy의 where 함수의 첫 번째 인수는 불리언 Series를 생성할 조건이다.

In [101]:
winner.style.highlight_min(axis=1)

Name,Amy,Bob,Winner
Month,,,
Apr,-0.053,-0.042,Amy
Feb,-0.089,-0.053,Amy
Jan,-0.036,-0.027,Amy
Mar,-0.017,-0.026,Bob


- pandas는 Month를 알파벳순으로 정렬한다.
- 이것은 월의 데이터 형식을 범주형으로 바꾸면 해결할 수 있다.
- 범주형 변수는 각 열의 모든 값을 정수로 매핑한다.









## 9.apply를 사용해 주별 SAT 가중 평균 구하기
## 10.연속 변수에 대한 그룹화
## 11.도시 간 총 비행 횟수 계산
## 12.최장 연속 정시 운행 찾기